# 7. Building a Regression MLP with Keras

## 7.1. De Data: California Housing

Voor regressie gebruiken we de **California Housing dataset** om huizenprijzen te voorspellen.

- **Kenmerken:** De dataset bevat enkel numerieke features en heeft geen ontbrekende waarden.
- **Laden:** We gebruiken `fetch_california_housing` van Scikit-Learn.

In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()

# Splitsen in training+validatie en test set
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)

# Splitsen van de training set in train en validatie
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

## 7.2. Architectuur en Configuratie

Een regressie MLP verschilt op een paar cruciale punten van een classifier:

- **Output Neuron:** Omdat we een enkele waarde voorspellen, gebruiken we slechts **één output neuron**.
- **Activatie:** De output laag heeft **geen activatiefunctie**.
- **Loss Functie:** We gebruiken **Mean Squared Error (MSE)**, wat standaard is voor regressie.
- **Optimizer:** In dit voorbeeld gebruiken we de **Adam** optimizer (een variant van SGD) met een learning rate van $10^{-3}$.
- **Hidden Layers:** Het model bevat 3 verborgen lagen met elk 50 neuronen en de ReLU-activatiefunctie.

## 7.3. Normalisatie met de `Normalization` Laag

In plaats van handmatige schaling (zoals `/ 255.` bij afbeeldingen), gebruiken we een Keras `Normalization` laag.

- **Functie:** Deze laag doet hetzelfde als de `StandardScaler` van Scikit-Learn (gemiddelde op 0, variantie op 1 brengen).
- **Belangrijk:** Je moet de methode `adapt()` aanroepen op de trainingsdata **voordat** je `fit()` aanroept. De laag berekent dan het gemiddelde en de variantie van elke feature.

## 4. Implementatie in Code

In [5]:
import keras

# 1. Normalisatie laag definiëren
norm_layer = keras.layers.Normalization()

# 2. Model opbouwen
model = keras.Sequential([
    keras.layers.Input(shape=X_train.shape[1:]),
    norm_layer,
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1) # Geen activatie voor regressie output
])

# 3. Compileren
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

# 4. Data aanpassen en trainen
norm_layer.adapt(X_train) # Bereken mean/variance voor schaling
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - RootMeanSquaredError: 0.9522 - loss: 0.9066 - val_RootMeanSquaredError: 0.7028 - val_loss: 0.4939
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - RootMeanSquaredError: 0.6118 - loss: 0.3743 - val_RootMeanSquaredError: 0.7640 - val_loss: 0.5836
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - RootMeanSquaredError: 0.5936 - loss: 0.3524 - val_RootMeanSquaredError: 0.5791 - val_loss: 0.3354
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - RootMeanSquaredError: 0.5787 - loss: 0.3349 - val_RootMeanSquaredError: 0.5815 - val_loss: 0.3382
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step - RootMeanSquaredError: 0.5741 - loss: 0.3295 - val_RootMeanSquaredError: 0.8223 - val_loss: 0.6761
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - RootMeanSquaredError: 0.5636 - loss: 0.3176 - val_RootMeanSquaredError: 0.5372 - val_loss: 0.2886
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - RootMeanSquaredError

## 5. Evaluatie en Voorspelling

Na de training evalueren we de prestaties op de testset en maken we voorspellingen voor nieuwe data.

In [6]:
# Evaluatie op test data
mse_test, rmse_test = model.evaluate(X_test, y_test)

# Voorspelling maken
X_new = X_test[:3]
y_pred = model.predict(X_new)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - RootMeanSquaredError: 0.5342 - loss: 0.2853
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
